In [2]:
import tabula
import PyPDF2

In [21]:


def extract_tables_from_pdf(file_path):
    # Open the PDF file
    pdf_file = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF
    num_pages = len(pdf_reader.pages)

    # Initialize an empty list to store the extracted tables
    tables = []

    # Iterate over each page in the PDF
    for page_num in range(num_pages):
        # Extract the tables from the current page
        page_tables = tabula.read_pdf(file_path, pages=page_num+1, multiple_tables=True)

        # Add the extracted tables to the list
        tables.extend(page_tables)

    # Close the PDF file
    pdf_file.close()

    return tables

# Example usage:
caminho_pdf = './files/old_pdfs/CCI1.088_2024.pdf' # './files/old_pdfs/CCI1.088_2024.pdf'
tables = extract_tables_from_pdf(caminho_pdf)

# Print the extracted tables
for table in tables:
    display(table)

,Data,Descrição,Responsável
0,Até 31/10/2024,Envio da planilha Plano de Ação 2025 por meio ...,Cooperativas centrais
1,Até 5/11/2024,Análiseevalidaçãodasplanilhasrecebidasno\rPort...,CCS


In [22]:
import fitz

doc = fitz.open('./files/old_pdfs/CCI1.088_2024.pdf')
for page in doc:
    tables = page.get_tables()
    for table in tables:
        print(table)

AttributeError: 'Page' object has no attribute 'get_tables'

In [1]:
import ollama

In [22]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings
from rich import print
from langchain.globals import set_debug
from langchain_core.pydantic_v1 import BaseModel, Field
from typing_extensions import Annotated, TypedDict, Optional
import json

set_debug(True)
    
model = ChatOllama(model="gemma2:2b-instruct-q4_K_M", keep_alive='1h', temperature=0)

with open('./files/pdfs/PDF.txt', 'r', encoding='utf-8') as f:
    context = f.read()
    response_schemas = [
        ResponseSchema(
            name="tipo_processo_classe_ou_classe_judicial",
            description="tipo_processo_ou_classe_classe_judicial usado para responder à pergunta do usuário, deve ser o Tipo de Processo, Classe ou Classe Judicial no processo.",
            type='string | \"não encontrado\"'
        ),
        ResponseSchema(
            name="estado_comarca",
            description="estado_comarca usado para responder à pergunta do usuário, deve ser o Estado ou Comarca no processo.",
            type='string | \"não encontrado\"'
        ),
        ResponseSchema(
            name="classificacao_de_prioridade",
            description="classificacao_de_prioridade usado para responder à pergunta do usuário, deve ser a Classificação de Prioridade no processo.",
            type='string | \"não encontrado\"'
        ),
        ResponseSchema(
            name="valor_da_causa",
            description="valor_da_causa usado para responder à pergunta do usuário, deve ser o Valor da causa no processo.",
            type='float | 0.0'
        ),
        ResponseSchema(
            name="orgao_julgador",
            description="orgao_julgador usado para responder à pergunta do usuário, deve ser órgão julgado do processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="numero_processo",
            description="numero_processo usado para responder à pergunta do usuário, deve ser um Número do processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="assuntos",
            description="assuntos usado para responder à pergunta do usuário, deve ser um Assunto no processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="nivel_de_sigilo",
            description="nivel_de_sigilo usado para responder à pergunta do usuário, deve ser o Nível de Sigilo do processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="justica_gratuita",
            description="justica_gratuita usado para responder à pergunta do usuário, se é gratuita a justiça no processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="pedido_tutela",
            description="pedido_tutela usado para responder à pergunta do usuário, deve ser o Pedido de liminar ou Antecipação de Tutela no processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="renda_liquida_mensal",
            description="renda_liquida_mensal usado para responder à pergunta do usuário, deve ser a Renda líquida mensal da parte autora no processo.",
            type="float | 0.0"
        ),
        ResponseSchema(
            name="partes",
            description="partes usado para responder à pergunta do usuário, deve ser as partes do processo.",
            type="List[string] | []"
        ),
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    format_instructions = output_parser.get_format_instructions(only_json=True)
    prompt = PromptTemplate(
        template="Responda às perguntas dos usuários da melhor maneira possível utilizando apenas o contexto fornecido.\n{format_instructions}\n{question}\n\n\n### Contexto ###\n\n```{context}```",
        input_variables=["question"],
        partial_variables={"format_instructions": format_instructions, "context": context},
    )
    chain = prompt | model | output_parser

    output = chain.invoke({"question": "Informe os dados do processo."})
    print(output)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Informe os dados do processo."
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "Informe os dados do processo."
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: Responda às perguntas dos usuários da melhor maneira possível utilizando apenas o contexto fornecido.\n\n```json\n{\n\t\"tipo_processo_classe_ou_classe_judicial\": string | \"não encontrado\"  // tipo_processo_ou_classe_classe_judicial usado para responder à pergunta do usuário, deve ser o Tipo de Processo, Classe ou Classe Judicial no processo.\n\t\"estado_comarca\": string | \"não encontrado\"  // estado_comarca usado para responder à pergunta do usuário, deve ser o Estado ou Comarca no processo.\n\t\"classifi

{
    'tipo_processo_classe_ou_classe_judicial': 'Procedimento Comum Cível',
    'estado_comarca': 'Distrito Federal e dos Territórios',
    'classificacao_de_prioridade': 'Não Informada',
    'valor_da_causa': 198133.18,
    'orgao_julgador': 'Vara Cível do Guará',
    'numero_processo': '0702763-79.2024.8.07.0014',
    'assuntos': 'Bancários',
    'nivel_de_sigilo': '0 (Público)',
    'justica_gratuita': 'NÃO',
    'pedido_tutela': 'SIM',
    'renda_liquida_mensal': 3110.6,
    'partes': [
        'MARISA DA COSTA BAPTISTA',
        'BANCO COOPERATIVO SICOOB S.A.',
        'EAGLE SOCIEDADE DE CREDITO DIRETO S.A.',
        'NU PAGAMENTOS S.A.'
    ]
}

In [ ]:
"You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",